### Get Electrophysiolgy Data

In [4]:
#Import the "Cell Types Cache" from the AllenSDK core package
from allensdk.core.cell_types_cache import CellTypesCache

#Import CellTypesApi, which will allow us to query the database.
from allensdk.api.queries.cell_types_api import CellTypesApi

# initialize cache as 'ctc' (cell types cache)
ctc = CellTypesCache(manifest_file='cell_types/manifest.json')

import pandas as pd
import numpy as np

Get ephys features and put into a dataframe. Make df of just id's since that is whats used to access the spike ephys data

In [5]:
ephys_features = ctc.get_ephys_features()
ephys_features_df = pd.DataFrame(ephys_features)
specimen_ids_df = ephys_features_df[['id']]
specimen_ids_df

,id
0,529951870
1,549196534
2,579986979
3,439816233
4,515191763
...,...
2328,569957302
2329,573464735
2330,576324611
2331,520471458


Copied over spike width calculation from example online

In [6]:
#def calc_spike_width(specimen_id):
# use this to get sweep
from allensdk.ephys.ephys_extractor import EphysSweepFeatureExtractor

specimen_id = 529951870     # try calculation with the first specimen in specimen_ids_df
data_set = ctc.get_ephys_data(specimen_id)
    
sweep_number = 35
sweep_data = data_set.get_sweep(sweep_number)

index_range = sweep_data["index_range"]
i = sweep_data["stimulus"][0:index_range[1]+1] # in A
v = sweep_data["response"][0:index_range[1]+1] # in V
i *= 1e12 # to pA
v *= 1e3 # to mV

sampling_rate = sweep_data["sampling_rate"] # in Hz
t = np.arange(0, len(v)) * (1.0 / sampling_rate)

sweep_ext = EphysSweepFeatureExtractor(t=t, v=v, i=i, start=1.02, end=2.02)
sweep_ext.process_spikes()

print(1e3 * np.nanmean(sweep_ext.spike_feature("width")))

Exception: Specimen 529951870 has no ephys data

^^^ gave us an error since 'Specimen 529951870 has no ephys data'.

The example used a difference specimen #. See if that specimen # is in our df to make sure we are using the correct set of specimen.

In [7]:
# checks if specimen id is in our df for our specimen at index 0
ephys_features_df[ephys_features_df['id'] == 529951870]

,adaptation,avg_isi,electrode_0_pa,f_i_curve_slope,fast_trough_t_long_square,fast_trough_t_ramp,fast_trough_t_short_square,fast_trough_v_long_square,fast_trough_v_ramp,fast_trough_v_short_square,...,trough_t_ramp,trough_t_short_square,trough_v_long_square,trough_v_ramp,trough_v_short_square,upstroke_downstroke_ratio_long_square,upstroke_downstroke_ratio_ramp,upstroke_downstroke_ratio_short_square,vm_for_sag,vrest
0,NaN,134.7,22.697498,0.083355,1.18768,13.2952,1.025916,-56.375004,-57.38542,-57.431251,...,13.29568,1.13478,-56.593754,-57.739586,-74.143753,3.029695,3.061646,2.969821,-80.46875,-73.553391


In [8]:
# checks if specimen used in example is in the df
ephys_features_df[ephys_features_df['id'] == 485909730]

,adaptation,avg_isi,electrode_0_pa,f_i_curve_slope,fast_trough_t_long_square,fast_trough_t_ramp,fast_trough_t_short_square,fast_trough_v_long_square,fast_trough_v_ramp,fast_trough_v_short_square,...,trough_t_ramp,trough_t_short_square,trough_v_long_square,trough_v_ramp,trough_v_short_square,upstroke_downstroke_ratio_long_square,upstroke_downstroke_ratio_ramp,upstroke_downstroke_ratio_short_square,vm_for_sag,vrest


The specimen used in the example isnt in the df we are currently using. Find a df that has the example specimen and that df should have more specimen that have ephys data and can use `.get_ephys_data()`.

Looked for other ways to get data from ctc : https://allensdk.readthedocs.io/en/latest/allensdk.core.cell_types_cache.html. Decided on using `.get_all_features()`

In [9]:
# use get all features and see if it has the cells with the ephys data
all_feats_df = pd.DataFrame(ctc.get_all_features())
all_ids = all_feats_df[['specimen_id']] 

# check if line below print line correctly asseses if a specimen is in the df
print(all_ids.loc[0])
all_ids[all_ids['specimen_id'] == 529878215]

specimen_id    529878215
Name: 0, dtype: int64


,specimen_id
0,529878215


In [12]:
# check if the specimen used in the example for calculating spike width is in the same df
all_ids[all_ids['specimen_id'] == 464212183]

,specimen_id
332,464212183


This df has the example specimen! That means we are in the right dataframe to start using `.get_ephys_data`!!

Check the 1st specimen id in the table to see if calculating spike width works

In [13]:
specimen_id = 529878215 
data_set = ctc.get_ephys_data(specimen_id)
    
sweep_number = 35
sweep_data = data_set.get_sweep(sweep_number)

index_range = sweep_data["index_range"]
i = sweep_data["stimulus"][0:index_range[1]+1] # in A
v = sweep_data["response"][0:index_range[1]+1] # in V
i *= 1e12 # to pA
v *= 1e3 # to mV

sampling_rate = sweep_data["sampling_rate"] # in Hz
t = np.arange(0, len(v)) * (1.0 / sampling_rate)

sweep_ext = EphysSweepFeatureExtractor(t=t, v=v, i=i, start=1.02, end=2.02)
sweep_ext.process_spikes()

print(1e3 * np.nanmean(sweep_ext.spike_feature("width")))

0.8600000000000829


/home/bdejong/.local/lib/python3.6/site-packages/allensdk/core/nwb_data_set.py:108: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  stimulus = stimulus_dataset.value
/home/bdejong/.local/lib/python3.6/site-packages/allensdk/core/nwb_data_set.py:109: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  response = swp['response']['timeseries']['data'].value
/home/bdejong/.local/lib/python3.6/site-packages/allensdk/core/nwb_data_set.py:125: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  swp_idx_start = swp['stimulus']['idx_start'].value
/home/bdejong/.local/lib/python3.6/site-packages/allensdk/core/nwb_data_set.py:126: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  swp_length = swp['stimulus']['count'].value
/home/bdejong/.local/lib/python3.6/site-packages/allensdk/core/nwb_data_set.py:135: H5pyDeprecationWarning: dataset.value has been depre

It works! 

Make the calculation into a helper function.

In [99]:
def calc_spike_width(specimen_id):
    
    data_set = ctc.get_ephys_data(specimen_id)

    sweep_number = 35
    sweep_data = data_set.get_sweep(sweep_number)

    index_range = sweep_data["index_range"]
    i = sweep_data["stimulus"][0:index_range[1]+1] # in A
    v = sweep_data["response"][0:index_range[1]+1] # in V
    i *= 1e12 # to pA
    v *= 1e3 # to mV

    sampling_rate = sweep_data["sampling_rate"] # in Hz
    t = np.arange(0, len(v)) * (1.0 / sampling_rate)

    sweep_ext = EphysSweepFeatureExtractor(t=t, v=v, i=i, start=1.02, end=2.02)
    sweep_ext.process_spikes()

    return round(1e3 * np.nanmean(sweep_ext.spike_feature("width")),4)

In [100]:
spike_width_ex = calc_spike_width(529878215)
print(spike_width_ex)

0.86


/home/bdejong/.local/lib/python3.6/site-packages/allensdk/ephys/ephys_extractor.py:175: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  spikes_df.ix[valid_ind, k + "_index"] = vals
/home/bdejong/.local/lib/python3.6/site-packages/allensdk/ephys/ephys_extractor.py:176: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  spikes_df.ix[valid_ind, k + "_t"] = t[vals]
/home/bdejong/.local/lib/python3.6/site-packages/allensdk/ephys/ephys_extractor.py:177: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stab

We will just ignore the red text for now...

Our function works. Now to apply it to all the specimen ids and make a new column for spike width.

In [106]:
#index num 14 and 15 has an issue
calc_spike_width(all_ids['specimen_id'][16])

KeyError: "Unable to open object (object 'Sweep_35' doesn't exist)"

^^ didnt work. sweep 35 doesnt exist for some (a lot) entries. Need to change the start times too.

In [105]:
# this gives a list of spike start times.
# the example focused on getting the 1st spike in a sweep so that's what we will do
spike_times = sweep_ext.spike_feature("threshold_t")
spike_times[0]

# but how do we get end time???

# we should also see if theres a way to get the last sweep frame, since that sweep frame will always have APs
# unlike some of the first sweeps

1.02306

2020-02-23 14:54:21,339 allensdk.api.api.retrieve_file_over_http INFO     Downloading URL: http://api.brain-map.org/api/v2/well_known_file_download/659450664


KeyError: "Unable to open object (object 'Sweep_35' doesn't exist)"

,specimen_id
0,529878215
1,548459652
2,479013100
3,567952169
4,569095789
5,528706755
6,508279351
7,643582610
8,464198958
9,582644266


In [86]:
a=[]
for i in all5['specimen_id']:
    spike_width_sing = calc_spike_width(i)
    a.append(spike_width_sing)

a

/home/bdejong/.local/lib/python3.6/site-packages/allensdk/ephys/ephys_extractor.py:175: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  spikes_df.ix[valid_ind, k + "_index"] = vals
/home/bdejong/.local/lib/python3.6/site-packages/allensdk/ephys/ephys_extractor.py:176: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  spikes_df.ix[valid_ind, k + "_t"] = t[vals]
/home/bdejong/.local/lib/python3.6/site-packages/allensdk/ephys/ephys_extractor.py:177: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stab

/home/bdejong/.local/lib/python3.6/site-packages/allensdk/ephys/ephys_extractor.py:198: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  spikes_df.ix[valid_ind, k + "_index"] = vals
/home/bdejong/.local/lib/python3.6/site-packages/allensdk/ephys/ephys_extractor.py:199: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  spikes_df.ix[valid_ind, k + "_t"] = t[vals]
/home/bdejong/.local/lib/python3.6/site-packages/allensdk/ephys/ephys_extractor.py:200: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stab

/home/bdejong/.local/lib/python3.6/site-packages/allensdk/ephys/ephys_extractor.py:198: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  spikes_df.ix[valid_ind, k + "_index"] = vals
/home/bdejong/.local/lib/python3.6/site-packages/allensdk/ephys/ephys_extractor.py:199: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  spikes_df.ix[valid_ind, k + "_t"] = t[vals]
/home/bdejong/.local/lib/python3.6/site-packages/allensdk/ephys/ephys_extractor.py:200: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stab

[0.86,
 nan,
 nan,
 nan,
 nan,
 1.86,
 0.95,
 0.3862,
 0.9875,
 nan,
 nan,
 0.8737,
 nan,
 nan]

In [82]:
# ends in 1048576?
list(all_ids['specimen_id'])

[529878215,
 548459652,
 479013100,
 567952169,
 569095789,
 528706755,
 508279351,
 643582610,
 464198958,
 582644266,
 501799874,
 569820026,
 497611660,
 535708196,
 542137330,
 596887423,
 586073683,
 475625477,
 478888052,
 487664663,
 554807924,
 478499902,
 539537094,
 478586295,
 576107598,
 510715606,
 569723367,
 571471295,
 611610514,
 613468354,
 527942865,
 571709783,
 571520182,
 485837504,
 479010903,
 508421103,
 471141261,
 314900022,
 512322162,
 313862167,
 585832440,
 502999078,
 526725807,
 573404307,
 476049169,
 596052605,
 480351780,
 571700636,
 580162374,
 386049446,
 397353539,
 548473051,
 475585413,
 318331342,
 501845152,
 323535089,
 329550137,
 488117124,
 574067499,
 545604343,
 548408551,
 571624532,
 486560376,
 485184849,
 567354967,
 591268268,
 478110866,
 530081951,
 485835016,
 529807751,
 589760138,
 529898751,
 480114344,
 530737765,
 515524026,
 567731428,
 583146274,
 562541627,
 574734127,
 476616076,
 565417112,
 333785962,
 476048909,
 471

In [44]:
calc_spike_width(548459652)

/home/bdejong/.local/lib/python3.6/site-packages/allensdk/core/nwb_data_set.py:108: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  stimulus = stimulus_dataset.value
/home/bdejong/.local/lib/python3.6/site-packages/allensdk/core/nwb_data_set.py:109: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  response = swp['response']['timeseries']['data'].value
/home/bdejong/.local/lib/python3.6/site-packages/allensdk/core/nwb_data_set.py:125: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  swp_idx_start = swp['stimulus']['idx_start'].value
/home/bdejong/.local/lib/python3.6/site-packages/allensdk/core/nwb_data_set.py:126: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  swp_length = swp['stimulus']['count'].value
/home/bdejong/.local/lib/python3.6/site-packages/allensdk/core/nwb_data_set.py:135: H5pyDeprecationWarning: dataset.value has been depre

nan

In [40]:
# df_users['zip'] = df_users['zip'].apply(helper)
all_ids['spike_width'] = all_ids['specimen_id'].apply(calc_spike_width)
spike_width_df = all_ids
spike_width_df.head()

/home/bdejong/.local/lib/python3.6/site-packages/allensdk/core/nwb_data_set.py:108: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  stimulus = stimulus_dataset.value
/home/bdejong/.local/lib/python3.6/site-packages/allensdk/core/nwb_data_set.py:109: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  response = swp['response']['timeseries']['data'].value
/home/bdejong/.local/lib/python3.6/site-packages/allensdk/core/nwb_data_set.py:125: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  swp_idx_start = swp['stimulus']['idx_start'].value
/home/bdejong/.local/lib/python3.6/site-packages/allensdk/core/nwb_data_set.py:126: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  swp_length = swp['stimulus']['count'].value
/home/bdejong/.local/lib/python3.6/site-packages/allensdk/core/nwb_data_set.py:135: H5pyDeprecationWarning: dataset.value has been depre

/home/bdejong/.local/lib/python3.6/site-packages/allensdk/ephys/ephys_extractor.py:175: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  spikes_df.ix[valid_ind, k + "_index"] = vals
/home/bdejong/.local/lib/python3.6/site-packages/allensdk/ephys/ephys_extractor.py:176: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  spikes_df.ix[valid_ind, k + "_t"] = t[vals]
/home/bdejong/.local/lib/python3.6/site-packages/allensdk/ephys/ephys_extractor.py:177: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stab

/home/bdejong/.local/lib/python3.6/site-packages/allensdk/ephys/ephys_extractor.py:175: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  spikes_df.ix[valid_ind, k + "_index"] = vals
/home/bdejong/.local/lib/python3.6/site-packages/allensdk/ephys/ephys_extractor.py:176: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  spikes_df.ix[valid_ind, k + "_t"] = t[vals]
/home/bdejong/.local/lib/python3.6/site-packages/allensdk/ephys/ephys_extractor.py:177: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stab

OSError: Unable to open file (truncated file: eof = 1048576, sblock->base_addr = 0, stored_eof = 8616008)

In [29]:
all_feats_df.drop(columns = ['total_length', 'total_surface', 'total_volume', 'superseded', 'avg_isi', 'electrode_0_pa',
                            'f_i_curve_slope','adaptation'])

,fast_trough_t_long_square,fast_trough_t_ramp,fast_trough_t_short_square,fast_trough_v_long_square,fast_trough_v_ramp,fast_trough_v_short_square,has_burst,has_delay,has_pause,id_x,...,number_stems,number_tips,overall_depth,overall_height,overall_width,scale_factor_x,scale_factor_y,scale_factor_z,soma_surface,tags
0,1.187680,13.295200,1.025916,-56.375004,-57.385420,-57.431251,False,False,False,529951870,...,7,63,135.5200,631.358581,581.674568,0.1144,0.1144,0.28,444.247484,3D Neuron Reconstruction morphology
1,1.099840,20.650105,1.025460,-54.000000,-54.828129,-54.656254,False,False,False,549196534,...,6,50,148.0957,819.531566,391.393464,0.1144,0.1144,0.28,1997.534304,3D Neuron Reconstruction morphology
2,1.190240,3.826338,1.025363,-44.937504,-43.635419,-47.395833,False,False,False,479020225,...,6,24,52.7875,406.386370,236.976305,0.1144,0.1144,0.28,522.952356,3D Neuron Reconstruction morphology
3,1.729100,16.728967,1.023988,-60.968754,-63.145835,-66.856253,False,False,False,567978743,...,7,30,57.2006,435.535475,332.948545,0.1144,0.1144,0.28,76.604264,3D Neuron Reconstruction morphology
4,1.109200,5.458287,1.025020,-51.062504,-57.916670,-56.531254,False,False,False,569112558,...,5,27,106.0998,256.025716,374.603996,0.1144,0.1144,0.28,77.936062,3D Neuron Reconstruction morphology
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
666,1.029680,16.606880,1.024076,-65.312508,-61.281254,-66.612503,False,False,False,396463317,...,7,32,112.1462,504.640357,358.994232,0.1144,0.1144,0.28,514.500853,3D Neuron Reconstruction morphology
667,1.191825,4.424122,1.024840,-47.750000,-47.281250,-51.416667,False,False,False,464359239,...,4,31,99.6856,392.731101,253.206728,0.1144,0.1144,0.28,304.439176,3D Neuron Reconstruction morphology
668,1.059920,13.743713,1.023850,-55.562504,-58.010419,-59.875002,False,False,False,643589290,...,5,17,83.6456,691.149333,557.317934,0.1144,0.1144,0.28,26.872017,3D Neuron Reconstruction morphology
669,1.119940,9.319540,1.024244,-56.500004,-56.208336,-60.012503,False,False,False,565472206,...,6,31,61.9624,364.482516,231.678905,0.1144,0.1144,0.28,148.866592,3D Neuron Reconstruction morphology


In [ ]:
ephys_features = ctc.get_ephys_features()
ephys_features_df = pd.DataFrame(ephys_features)
specimen_ids_df = ephys_features_df[['id']]
specimen_ids_df

In [ ]:
ephys_features_df_index_spec = ephys_features_df
ephys_features_df_index_spec = ephys_features_df_index_spec[['id','rheobase_sweep_id', 'rheobase_sweep_number','thumbnail_sweep_id']]
#ephys_features_df_index_spec.loc[464212183]
ephys_features_df_index_spec
464212183 in list(ephys_features_df_index_spec['id']) # the specimen used in the exmaple is not in our dataset....

It looks like sweep number is just any sweep where an AP does not result from the stimulus?
Ephys data from specimen 464212183 used in the example: http://celltypes.brain-map.org/mouse/experiment/electrophysiology/464212183.

What is the significance of the sweep number?????? Is there a relation to rheobase_sweep_number? (No? rheobase sweep number isnt displayed in the data on the link)

Get sweep number 30 for all cells to be safe

In [ ]:
#def calc_spike_width(specimen_id):
    # code taken from the source on spike calculation
specimen_id = 439816233
data_set = ctc.get_ephys_data(specimen_id)
    
sweep_number = 35
sweep_data = data_set.get_sweep(sweep_number)

index_range = sweep_data["index_range"]
i = sweep_data["stimulus"][0:index_range[1]+1] # in A
v = sweep_data["response"][0:index_range[1]+1] # in V
i *= 1e12 # to pA
v *= 1e3 # to mV

sampling_rate = sweep_data["sampling_rate"] # in Hz
t = np.arange(0, len(v)) * (1.0 / sampling_rate)

sweep_ext = EphysSweepFeatureExtractor(t=t, v=v, i=i, start=1.02, end=2.02)
sweep_ext.process_spikes()

print(1e3 * np.nanmean(sweep_ext.spike_feature("width")))


In [ ]:
helper(515191763)

Need to find specimens that have ephys data!!!!

In [ ]:
# get data for a single specimen ephys data
# electrophysiology traces for a single cell in the database
from allensdk.ephys.extract_cell_features import extract_cell_features


# pick a cell to analyze
specimen_id = 324257146

# download the ephys data and sweep metadata
data_set = ctc.get_ephys_data(specimen_id)
sweeps = ctc.get_ephys_sweeps(specimen_id)

sweep_number = 35
sweep_data = data_set.get_sweep(sweep_number)
sweep_data

### use .apply for get_ephys_data and put it into a column?
### use .apply for sweeps and put it int a column


### Calculate Spike Width